<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №8


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

**Описание задачи:**  
Создать базовый класс ***CreditCard*** в C#, который будет представлять информацию
о кредитных картах. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.
     
**Требования к базовому классу CreditCard:**  
* Атрибуты: Номер карты (***CardNumber***), Холдера (***HolderName***), Срок действия (***ExpiryDate***).  
* Методы:  
    * ***GetInfo()***: метод для получения информации о кредитной карте в виде строки.  
    * ***Pay()***: метод для оплаты покупки с использованием карты.  
    * ***CheckBalance()***: метод для проверки баланса на карте.  
  
**Требования к производным классам:**
* GoldCard (***GoldCreditCard***): ДДолжен содержать дополнительные атрибуты,
такие как Бесплатные бонусные мили (***BonusMiles***). Метод *Pay()* должен быть переопределен для
добавления информации о получении бонусных миль при оплате покупки.
* PremiumCard (***PremiumCreditCard***): Должен содержать дополнительные
атрибуты, такие как Ассистент поддержки (***SupportAssistant***). Метод *CheckBalance()* долженбыть переопределен для 
предоставления возможности обратиться за помощью к ассистенту поддержки в случае 
проблем с балансом.
* CorporateCard (***CorporateCreditCard***) (если требуется третий класс):
Должен содержать дополнительные атрибуты, такие как Компания
(***Company***). Метод *GetInfo()* должен быть переопределен для включения
информации в описании карты.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
public delegate void CardEventHandler(string message);

public class CreditCard
{
    public event CardEventHandler LowBalanceEvent;
    public event CardEventHandler PaymentSuccessEvent;

    public string CardNumber { get; set; }
    public string HolderName { get; set; }
    public string ExpiryDate { get; set; }
    public decimal Balance { get; protected set; }
    public string Currency { get; set; }
    public string Bank { get; set; }

    public DateTime CreatedDate { get; set; }
    public bool IsActive { get; private set; }
    public string Country { get; set; }
    public string CardType { get; set; }

    public CreditCard(string cardNumber, string holderName, string expiryDate, decimal balance)
    {
        CardNumber = cardNumber;
        HolderName = holderName;
        ExpiryDate = expiryDate;
        Balance = balance;
        Currency = "RUB";
        Bank = "Default Bank";
        Country = "Russia";
        CreatedDate = DateTime.Now;
        IsActive = true;
        CardType = "Basic";
    }

    public virtual string GetInfo()
    {
        return $"[{CardType}] {CardNumber} - {HolderName}, Баланс: {Balance} {Currency}";
    }

    public virtual void Pay(decimal amount)
    {
        if (!IsActive)
        {
            Console.WriteLine("Карта неактивна!");
            return;
        }

        if (amount <= Balance)
        {
            Balance -= amount;
            PaymentSuccessEvent?.Invoke($"Успешная оплата на сумму {amount}. Остаток: {Balance}");
            if (Balance < 1000)
            {
                LowBalanceEvent?.Invoke("Внимание: низкий баланс на карте!");
            }
        }
        else
        {
            Console.WriteLine("Недостаточно средств.");
        }
    }

    public void Activate() => IsActive = true;
    public void Deactivate() => IsActive = false;

    public virtual void CheckBalance() => Console.WriteLine($"Баланс: {Balance} {Currency}");
}


In [2]:
public class GoldCreditCard : CreditCard
{
    public int BonusPoints { get; set; }
    public decimal CashbackPercent { get; set; }
    public string LoungeAccessProvider { get; set; }
    public List<string> PurchaseHistory { get; set; }

    public GoldCreditCard(string cardNumber, string holderName, string expiryDate, decimal balance)
        : base(cardNumber, holderName, expiryDate, balance)
    {
        BonusPoints = 0;
        CashbackPercent = 2;
        LoungeAccessProvider = "Priority Pass";
        PurchaseHistory = new List<string>();
        CardType = "Gold";
    }

    public override void Pay(decimal amount)
    {
        base.Pay(amount);
        int bonus = (int)(amount / 50);
        BonusPoints += bonus;
        PurchaseHistory.Add($"Покупка на {amount} - начислено {bonus} бонусов");
    }

    public void ShowPurchaseHistory()
    {
        Console.WriteLine("История покупок:");
        foreach (var p in PurchaseHistory)
            Console.WriteLine($"- {p}");
    }

    public void AccessLounge()
    {
        Console.WriteLine($"Доступ в зал ожидания от {LoungeAccessProvider}");
    }
}


In [3]:
public class CardManager
{
    private Dictionary<string, CreditCard> cards = new Dictionary<string, CreditCard>();

    public void AddCard(CreditCard card)
    {
        if (!cards.ContainsKey(card.CardNumber))
        {
            cards.Add(card.CardNumber, card);
            Console.WriteLine($"Карта {card.CardNumber} добавлена.");
        }
        else
        {
            Console.WriteLine("Карта уже существует.");
        }
    }

    public void ShowAllCards()
    {
        Console.WriteLine("Список всех карт:");
        foreach (var card in cards.Values)
        {
            Console.WriteLine(card.GetInfo());
        }
    }

    public CreditCard GetCard(string number)
    {
        return cards.TryGetValue(number, out var card) ? card : null;
    }
}


In [4]:
GoldCreditCard gold = new GoldCreditCard("5555 9999 0000 1111", "Алексей", "10/26", 5000);

// Подписка на события
gold.LowBalanceEvent += msg => Console.WriteLine("СОБЫТИЕ: " + msg);
gold.PaymentSuccessEvent += msg => Console.WriteLine("СОБЫТИЕ: " + msg);

CardManager manager = new CardManager();
manager.AddCard(gold);

// Использование методов
gold.Pay(1000);
gold.Pay(4000);  // вызовет LowBalanceEvent
gold.CheckBalance();
gold.ShowPurchaseHistory();
gold.AccessLounge();

Console.WriteLine();

manager.ShowAllCards();

Карта 5555 9999 0000 1111 добавлена.
СОБЫТИЕ: Успешная оплата на сумму 1000. Остаток: 4000
СОБЫТИЕ: Успешная оплата на сумму 4000. Остаток: 0
СОБЫТИЕ: Внимание: низкий баланс на карте!
Баланс: 0 RUB
История покупок:
- Покупка на 1000 - начислено 20 бонусов
- Покупка на 4000 - начислено 80 бонусов
Доступ в зал ожидания от Priority Pass

Список всех карт:
[Gold] 5555 9999 0000 1111 - Алексей, Баланс: 0 RUB
